In [ ]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datasets import load_dataset
import webbrowser

# Cargar el dataset desde Hugging Face
dataset = load_dataset("Frorozcol/recetas-cocina")
df_train = pd.DataFrame(dataset['train'])

# Eliminar columna 'uuid' y eliminar filas con valores nulos
df_train = df_train.drop('uuid', axis=1)
df_train = df_train.dropna()

# Vectorizador TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_train['ingredients'].fillna(''))

# Función para buscar recetas por ingredientes
def buscar_recetas_por_ingredientes(df, lista_ingredientes):
    # Convertir los ingredientes a minúsculas
    lista_ingredientes = [ing.lower().strip() for ing in lista_ingredientes]
    
    # Filtrar las recetas que contienen todos los ingredientes proporcionados
    def contiene_todos_ingredientes(ingredientes):
        return all(ing in ingredientes.lower() for ing in lista_ingredientes)
    
    df_filtrado = df[df['ingredients'].apply(contiene_todos_ingredientes)]
    
    if df_filtrado.empty:
        return pd.DataFrame()  # Retorna un DataFrame vacío si no se encuentran recetas
    
    # Transformar los ingredientes del usuario usando el vectorizador TF-IDF
    ingredientes_usuario = ', '.join(lista_ingredientes)
    user_tfidf = vectorizer.transform([ingredientes_usuario])
    
    # Calcular la similitud del coseno entre los ingredientes del usuario y las recetas en el dataset filtrado
    tfidf_matrix_filtrado = vectorizer.transform(df_filtrado['ingredients'].fillna(''))
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix_filtrado).flatten()
    
    # Obtener los índices de las recetas más similares
    similar_indices = cosine_similarities.argsort()[-5:][::-1]
    
    # Obtener las recetas más similares
    recetas_recomendadas = df_filtrado.iloc[similar_indices]
    
    return recetas_recomendadas

def mostrar_recetas_en_ventana(recetas):
    ventana = tk.Toplevel()
    ventana.title("Recetas Recomendadas")
    bg_image = Image.open(r'C:\Users\weyel\Documents\samsumg_proyecto_final\img\sub1.png')  #cambia la ruta para donde se guarda la imagen en tu PC (primera ventana)
    photo = ImageTk.PhotoImage(bg_image)
    bg_label = tk.Label(ventana, image=photo)
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)
    bg_label.image = photo  # Mantener referencia

    for index, receta in recetas.iterrows():
        frame = ttk.Frame(ventana, padding="10", style='My.TFrame')
        frame.pack(fill='x', padx=10, pady=5)
        title_label = ttk.Label(frame, text=receta['title'], font=('Arial', 14, 'bold'), foreground='#B67352')
        title_label.pack(anchor='w', pady=5)
        url_label = ttk.Label(frame, text=receta['url'], foreground="blue", cursor="hand2", font=('Arial', 10))
        url_label.pack(anchor='w', pady=5)
        url_label.bind("<Button-1>", lambda e, url=receta['url']: abrir_url(url))

def abrir_url(url):
    webbrowser.open_new(url)

def open_ingredient_window():
    num_ingredients = int(ingredient_count.get())
    ingredient_window = tk.Toplevel(root)
    ingredient_window.title("Ingrese los Ingredientes")
    entries = []
    for i in range(num_ingredients):
        label = ttk.Label(ingredient_window, text=f'Ingrediente {i+1}:')
        label.pack()
        entry = ttk.Entry(ingredient_window, width=50)
        entry.pack()
        entries.append(entry)
    submit_button = ttk.Button(ingredient_window, text="Buscar Recetas", command=lambda: fetch_recipes(entries))
    submit_button.pack(pady=20)

def fetch_recipes(entries):
    ingredients = [entry.get() for entry in entries]
    recetas_recomendadas = buscar_recetas_por_ingredientes(df_train, ingredients)
    mostrar_recetas_en_ventana(recetas_recomendadas)

root = tk.Tk()
root.title("Recomendador de Recetas")
root.geometry('800x600')  # tamaño de la ventana (debe concider con los pixeles de la foto)

# Configurar y mostrar la imagen de fondo
bg_image = Image.open(r'C:\Users\weyel\Documents\samsumg_proyecto_final\img\principal1.png') #cambia la ruta para donde se guarda la imagen en tu PC (tercera ventana)
photo = ImageTk.PhotoImage(bg_image)
bg_label = tk.Label(root, image=photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
bg_label.image = photo  # Mantener referencia

# Configuración de los widgets sobre la imagen
ingredient_count = ttk.Spinbox(root, from_=1, to=6, width=5)
ingredient_count.place(x=350, y=500)  # Ajusta la posición según necesites

count_button = ttk.Button(root, text="Confirmar cantidad de ingredientes", command=open_ingredient_window)
count_button.place(x=350, y=530)  # Ajusta la posición según necesites

root.mainloop()
